In [3]:
%cd ..

/Users/katyscott/Documents/BHKLab_GitHub/readii_2_roqc


In [1]:
from pathlib import Path
from typing import Optional

import pandas as pd
import SimpleITK as sitk
from damply import dirs
from readii.image_processing import alignImages, flattenImage
from readii.io.loaders import loadImageDatasetConfig
from readii.io.writers.nifti_writer import NIFTIWriter, NiftiWriterIOError
from readii.negative_controls_refactor import NegativeControlManager
from readii.process.config import get_full_data_name
from readii.utils import logger

In [2]:
def get_readii_settings(dataset_config: dict) -> tuple[list, list, list]:
    """Extract READII settings from a configuration dictionary.
    
    Parameters
    ----------
    dataset_config : dict
        Configuration dictionary read in with `loadImageDatasetConfig` containing READII settings
    
    Returns
    -------
    tuple
        A tuple containing:
        - regions: list of regions to process
        - permutations: list of permutations to apply
        - crop: list of crop settings
    """
    readii_config = dataset_config['READII']
    if 'IMAGE_TYPES' not in readii_config:
        message = "READII configuration must contain 'IMAGE_TYPES'."
        logger.error(message)
        raise KeyError(message)
    
    regions = readii_config['IMAGE_TYPES']['regions']

    permutations = readii_config['IMAGE_TYPES']['permutations']

    crop = readii_config['IMAGE_TYPES']['crop']

    return regions, permutations, crop

In [11]:
def get_masked_image_metadata(dataset_index:pd.DataFrame,
                              dataset_config:Optional[dict] = None,
                              image_modality:Optional[str] = None,
                              mask_modality:Optional[str] = None):
    """Get rows of Med-ImageTools index.csv with the mask modality and the corresponding image modality and create a new index with just these rows for READII
    
    Parameters
    ----------
    dataset_index : pd.DataFrame
        DataFrame loaded from a Med-ImageTools index.csv containing image metadata. Must have columns for Modality, ReferencedSeriesUID, and SeriesInstanceUID.
    dataset_config : Optional[dict]
        Dictionary of configuration settings to get image and mask modality from for filtering dataset_index. Must include MIT MODALITIES image and MIT MODALITIES mask. Expected output from running loadImageDatasetConfig.
    image_modality : Optional[str]
        Image modality to filter dataset_index with. Will override dataset_config setting.
    mask_modality : Optional[str]
        Mask modality to filter dataset_index with. Will override dataset_config setting.

    Returns
    -------
    pd.DataFrame
        Subset of the dataset_index with just the masks and their reference images' metadata.
    """

    if image_modality is None:
        if dataset_config is None:
            message = "No image modality setting passed. Must pass a image_modality or dataset_config with an image modality setting."
            logger.error(message)
            raise ValueError(message)
        
        # Get the image modality from config to retrieve from the metadata
        image_modality = dataset_config["MIT"]["MODALITIES"]["image"]
    
    if mask_modality is None:
        if dataset_config is None:
            message = "No mask modality setting passed. Must pass a mask_modality or dataset_config with a mask modality setting."
            logger.error(message)
            raise ValueError(message)
        
        # Get the mask modality from config to retrieve from the metadata
        mask_modality = dataset_config["MIT"]["MODALITIES"]["mask"]

    # Get all metadata rows with the mask modality
    mask_metadata = dataset_index[dataset_index['Modality'] == mask_modality]

    # Get a Series of ReferenceSeriesUIDs from the masks - these point to the images the masks were made on
    referenced_series_ids = mask_metadata['ReferencedSeriesUID']
    
    # Get image metadata rows with a SeriesInstanceUID matching one of the ReferenceSeriesUIDS of the masks
    image_metadata = dataset_index[dataset_index['Modality'] == image_modality]
    masked_image_metadata = image_metadata[image_metadata['SeriesInstanceUID'].isin(referenced_series_ids)]

    # Return the subsetted metadata
    return pd.concat([masked_image_metadata, mask_metadata], sort=True)

In [12]:
def save_out_negative_controls(nifti_writer: NIFTIWriter,
                            patient_id: str,
                            image: sitk.Image,
                            region: str,
                            permutation: str):
    """Save out negative control images using the NIFTIWriter."""

    try:
        nifti_writer.save(
                        image,
                        PatientID=patient_id,
                        region=region,
                        permutation=permutation
                    )
    except NiftiWriterIOError:
        message = f"{permutation} {region} negative control file already exists for {patient_id}. If you wish to overwrite, set overwrite to true in the NIFTIWriter."
        logger.debug(message)

    return image

In [ ]:
dataset = "NSCLC-Radiomics"
random_seed = 10

if dataset is None:
    message = "Dataset name must be provided."
    logger.error(message)
    raise ValueError(message)

config_dir_path = dirs.CONFIG / 'datasets'

dataset_config = loadImageDatasetConfig(dataset, config_dir_path)

dataset_name = dataset_config['DATASET_NAME']
full_data_name = get_full_data_name(config_dir_path / dataset)
logger.info(f"Creating negative controls for dataset: {dataset_name}")

# Extract READII settings
regions, permutations, _crop = get_readii_settings(dataset_config)

# Set up negative control manager with settings from config
manager = NegativeControlManager.from_strings(
    negative_control_types=permutations,
    region_types=regions,
    random_seed=random_seed
)

mit_images_dir_path = dirs.PROCDATA / full_data_name / 'images' /f'mit_{dataset_name}'

dataset_index = pd.read_csv(Path(mit_images_dir_path, f'mit_{dataset_name}_index.csv'))

image_modality = dataset_config["MIT"]["MODALITIES"]["image"]
mask_modality = dataset_config["MIT"]["MODALITIES"]["mask"]

masked_image_index = get_masked_image_metadata(dataset_index = dataset_index,
                                                image_modality = image_modality,
                                                mask_modality = mask_modality)

In [ ]:
# StudyInstanceUID
for study, study_data in masked_image_index.groupby('StudyInstanceUID'):
    logger.info(f"Processing StudyInstanceUID: {study}")

    # Get image metadata as a pd.Series
    image_metadata = study_data[study_data['Modality'] == image_modality].squeeze()
    image_path = Path(image_metadata['filepath'])
    # Load in image
    raw_image = sitk.ReadImage(mit_images_dir_path / image_path)
    # Remove extra dimension of image, set origin, spacing, direction to original
    image = alignImages(raw_image, flattenImage(raw_image))

    
    # Get mask metadata as a pd.Series
    # all_mask_metadata = study_data[study_data['Modality'] == mask_modality]

    # for mask_metadata in all_mask_metadata.iterrows():
    #     mask_path = Path(mask_metadata['filepath'])
    #     # Load in mask
    #     raw_mask = sitk.ReadImage(mit_images_dir_path / mask_path)
    #     mask = alignImages(raw_mask, flattenImage(raw_mask))

    #     # Set up writer for saving out the negative controls
    #     nifti_writer = NIFTIWriter(
    #         root_directory = mit_images_dir_path.parent / f'readii_{dataset_name}' / image_path.parent / mask_path.parent,
    #         filename_format = "{permutation}_{region}.nii.gz",
    #         overwrite = True,
    #         create_dirs = True
    #     )

In [19]:
masked_image_index

,AcquisitionDate,AcquisitionNumber,AcquisitionTime,BodyPartExamined,CTDIvol,ContentDate,ContentTime,ContrastBolusAgent,ContrastBolusIngredient,ContrastBolusIngredientConcentration,...,ndim,nvoxels,origin,roi_key,saved_time,size,spacing,std,sum,variance
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,29097984,"(-250.112, -250.112, -133.4)",GTV,2025-05-29:17-31-31,"(512, 512, 111)","(0.977, 0.977, 3.0)",0.065499,125373.0,0.004290
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,35127296,"(-249.51171875, -460.51171875, -681.5)",GTV,2025-05-29:17-31-34,"(512, 512, 134)","(0.9765625, 0.9765625, 3.0)",0.039992,56271.0,0.001599


In [6]:
image_metadata['filepath']

'LUNG1-001_0000/CT_63382046/CT.nii.gz'

In [9]:
image.GetSpacing()

(1.0, 1.0, 1.0)

In [10]:
raw_image = sitk.ReadImage(mit_images_dir_path / image_path)
raw_image.GetSpacing()

(0.9765625, 0.9765625, 3.0)

In [14]:
neg_image.GetSpacing()

(0.9765625, 0.9765625, 3.0)

# Update readii index checking for overwrite

In [37]:
import itertools
readii_index_file = Path("/Users/katyscott/Documents/BHKLab_GitHub/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics/images/readii_NSCLC-Radiomics/readii_NSCLC-Radiomics_index.csv")
overwrite = False

dataset_config = loadImageDatasetConfig("NSCLC-Radiomics.yaml", "/Users/katyscott/Documents/BHKLab_GitHub/readii_2_roqc/config/datasets")

regions, permutations, _crop = get_readii_settings(dataset_config)
seed = 10
# Set up negative control manager with settings from config
manager = NegativeControlManager.from_strings(
    negative_control_types=permutations,
    region_types=regions,
    random_seed=seed
    )


if readii_index_file.exists() and not overwrite: 
    # Load in readii index and check:
    # 1. if all negative controls requested have been extracted
    # 2. for all of the patients
    readii_index = pd.read_csv(readii_index_file)

    processed_image_types = {itype for itype in readii_index[['Permutation', 'Region']].itertuples(index=False, name=None)}
    requested_image_types = {itype for itype in itertools.product([permutation.name() for permutation in manager.negative_control_strategies],
                                                                  [region.name() for region in manager.region_strategies])}
    
    # Check if the requested image types are a subset of those already processed
    if requested_image_types.issubset(processed_image_types):
        print("Requested negative controls have already been generated for these samples or are listed in the readii index as if they have been. Set overwrite to true if you want to re-process these.")
    

Requested negative controls have already been generated for these samples or are listed in the readii index as if they have been. Set overwrite to true if you want to re-process these.


# Filter med-imagetools simple-index with ROI regex

In [53]:
def roi_filter_mask_metadata(mask_metadata:pd.DataFrame,
                         dataset_config:dict
                         ):
    # Get the metadata rows with the specified ROIs - here for when autopipeline is run with more ROIs set to be extracted than want to be processed with R2R
    roi_regex = dataset_config["MIT"]["ROI_MATCH_MAP"]
    roi_strategy = dataset_config["MIT"]["ROI_STRATEGY"]
    mask_modality = dataset_config["MIT"]["MODALITIES"]["mask"]

    roi_regex_list = roi_regex.split(":")
    roi_key = roi_regex_list[0]
    roi_matches = roi_regex_list[1].strip("'").split(",")

    match roi_strategy:
        case 'MERGE':
            filtered_mask_metadata = mask_metadata[(mask_metadata['ImageID'] == roi_key) & (mask_metadata['Modality'] == mask_modality)] 
        case 'SEPARATE':
            # Get the combined roi key and match in the format generated by autopipeline separate = {roi_key}__{roi_items} with necessary escape characters for regex
            image_id_regex_combos = {rf"{roi_key}__\[{item}\]" for item in roi_matches}
            # Join each combination into a single regex string, separated by | for or
            image_id_regex = r"|".join(image_id_regex_combos)
            # Set a copy of the ImageID column as the index for the filtering function to use
            filtered_mask_metadata = mask_metadata.set_index('ImageID', drop=False)
            # Filter the dataset based on the regex string generated from the ROI Match Map
            filtered_mask_metadata = filtered_mask_metadata.filter(regex=image_id_regex, axis=0)
            # Restore the numeric index
            filtered_mask_metadata.reset_index(drop=True)
            filtered_mask_metadata = filtered_mask_metadata[(filtered_mask_metadata['Modality'] == mask_modality)]
        case _:
            message = f"The roi strategy {roi_strategy} is not handled by this function. No filtering will be applied."
            logger.info(message)
            filtered_mask_metadata = mask_metadata
            

    return filtered_mask_metadata

In [54]:
dataset_config = loadImageDatasetConfig("NSCLC-Radiomics.yaml", "/Users/katyscott/Documents/BHKLab_GitHub/readii_2_roqc/config/datasets")
dataset_index = pd.read_csv("/Users/katyscott/Documents/BHKLab_GitHub/readii_2_roqc/data/procdata/TCIA_NSCLC-Radiomics/images/mit_NSCLC-Radiomics/mit_NSCLC-Radiomics_index-simple.csv")

roi_filter_mask_metadata(dataset_index, dataset_config)


,filepath,hash,saved_time,SampleNumber,ImageID,PatientID,Modality,SeriesInstanceUID,StudyInstanceUID,ReferencedSeriesUID,...,mask.bbox.min_coord,mask.bbox.max_coord,mask.feret_diameter,mask.roundness,mask.flatness,mask.elongation,mask.equivalent_spherical_radius,mask.equivalent_spherical_perimeter,mask.equivalent_ellipsoid_diameters,mask.volume_count
ImageID,,,,,,,,,,,,,,,,,,,,,
ROI__[GTV-1],LUNG1-001_0000/RTSTRUCT_35578236/ROI__[GTV-1]....,7afa5f960c8113372e0ad0a8e8303f7eaea396f4,2025-07-24:20:06:02,0,ROI__[GTV-1],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(290, 226, 65)","(388, 317, 86)",95.647976,0.634964,1.335891,1.367931,33.747307,14311.597300,"(50.12645370811163, 66.96349744548279, 91.6014...",1.0
ROI__[Spinal-Cord],LUNG1-001_0000/RTSTRUCT_35578236/ROI__[Spinal-...,cf1c2a5eb41e692152b3288edf6e6ed22cfec98c,2025-07-24:20:06:03,0,ROI__[Spinal-Cord],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(250, 280, 27)","(278, 369, 111)",259.871631,0.560548,2.849275,6.132994,22.899470,6589.625518,"(12.449207360728114, 35.47121071739851, 217.54...",1.0
ROI__[GTV-1],LUNG1-002_0001/RTSTRUCT_43245931/ROI__[GTV-1]....,cdfb2786c29b57287cc85233f0c02df51427b472,2025-07-24:20:06:20,1,ROI__[GTV-1],LUNG1-002,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2432675512669112458302...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,...,"(129, 226, 28)","(229, 325, 54)",123.299408,0.731734,1.167976,1.133250,44.090186,24428.326986,"(76.26187390348498, 89.07202002328074, 100.940...",1.0
ROI__[Spinal-Cord],LUNG1-002_0001/RTSTRUCT_43245931/ROI__[Spinal-...,3cab461010443ebd203a250988ce6e6a45d04b8f,2025-07-24:20:06:19,1,ROI__[Spinal-Cord],LUNG1-002,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2432675512669112458302...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,...,"(251, 235, 0)","(280, 334, 111)",339.654273,0.504264,3.285958,6.526126,24.922869,7805.593481,"(12.067986928374351, 39.65490376055276, 258.79...",1.0


In [ ]:

roi_regex = dataset_config["MIT"]["ROI_MATCH_MAP"]
roi_strategy = dataset_config["MIT"]["ROI_STRATEGY"]

roi_regex_list = roi_regex.split(":")
roi_key = roi_regex_list[0]
roi_items = roi_regex_list[1].strip("'").split(",")

,filepath,hash,saved_time,SampleNumber,ImageID,PatientID,Modality,SeriesInstanceUID,StudyInstanceUID,ReferencedSeriesUID,...,mask.bbox.min_coord,mask.bbox.max_coord,mask.feret_diameter,mask.roundness,mask.flatness,mask.elongation,mask.equivalent_spherical_radius,mask.equivalent_spherical_perimeter,mask.equivalent_ellipsoid_diameters,mask.volume_count
0,LUNG1-001_0000/CT_63382046/CT.nii.gz,b4175e0d6ddff17507e30c05c6e7302c2b9668f4,2025-07-15:21:13:49,0,CT,LUNG1-001,CT,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LUNG1-001_0000/RTSTRUCT_35578236/GTV.nii.gz,7afa5f960c8113372e0ad0a8e8303f7eaea396f4,2025-07-15:21:13:52,0,GTV,LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(290, 226, 65)","(388, 317, 86)",95.647976,0.634964,1.335891,1.367931,33.747307,14311.597300,"(50.12645370811163, 66.96349744548279, 91.6014...",1.0
2,LUNG1-001_0000/RTSTRUCT_35578236/ROI__[GTV-1]....,7afa5f960c8113372e0ad0a8e8303f7eaea396f4,2025-07-24:20:06:02,0,ROI__[GTV-1],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(290, 226, 65)","(388, 317, 86)",95.647976,0.634964,1.335891,1.367931,33.747307,14311.597300,"(50.12645370811163, 66.96349744548279, 91.6014...",1.0
3,LUNG1-001_0000/RTSTRUCT_35578236/ROI__[Lung-Le...,68379d7b721f4a7b9960420dad78db01c1de7de8,2025-07-24:20:06:13,0,ROI__[Lung-Left],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(265, 164, 24)","(417, 370, 112)",281.846234,0.539037,1.593719,1.411149,82.059400,84618.736906,"(107.24066680307503, 170.91153991427942, 241.1...",3.0
4,LUNG1-001_0000/RTSTRUCT_35578236/ROI__[Lung-Ri...,caa262a733f87f1a66b8063a81a28bb1a720df70,2025-07-24:20:06:24,0,ROI__[Lung-Right],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(110, 160, 23)","(273, 380, 109)",278.662362,0.661106,1.615824,1.075100,89.531955,100731.661088,"(126.9391936225655, 205.1114141714691, 220.515...",6.0
5,LUNG1-001_0000/RTSTRUCT_35578236/ROI__[Spinal-...,cf1c2a5eb41e692152b3288edf6e6ed22cfec98c,2025-07-24:20:06:03,0,ROI__[Spinal-Cord],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(250, 280, 27)","(278, 369, 111)",259.871631,0.560548,2.849275,6.132994,22.899470,6589.625518,"(12.449207360728114, 35.47121071739851, 217.54...",1.0
6,LUNG1-001_0000/RTSTRUCT_35578236/Spinal-Cord.n...,cf1c2a5eb41e692152b3288edf6e6ed22cfec98c,2025-07-24:20:07:07,0,Spinal-Cord,LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(250, 280, 27)","(278, 369, 111)",259.871631,0.560548,2.849275,6.132994,22.899470,6589.625518,"(12.449207360728114, 35.47121071739851, 217.54...",1.0
7,LUNG1-002_0001/CT_23261228/CT.nii.gz,94c6d14e8cf0b758c0cffad057ab2b239a93ee16,2025-07-15:21:13:48,1,CT,LUNG1-002,CT,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LUNG1-002_0001/RTSTRUCT_43245931/GTV.nii.gz,cdfb2786c29b57287cc85233f0c02df51427b472,2025-07-15:21:13:50,1,GTV,LUNG1-002,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2432675512669112458302...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,...,"(129, 226, 28)","(229, 325, 54)",123.299408,0.731734,1.167976,1.133250,44.090186,24428.326986,"(76.26187390348498, 8

In [31]:
# SEPARATE
# Will be {roi_key}__{roi_items}
image_id_regex_combos = {rf"{roi_key}__\[{item}\]" for item in roi_items}

image_id_regex = r"|".join(image_id_regex_combos)
image_id_regex

'ROI__\\[Spinal-Cord\\]|ROI__\\[GTV-1\\]'

In [39]:
image_index = dataset_index.set_index('ImageID', drop=False)
filtered_dataset = image_index.filter(regex=image_id_regex, axis=0)
filtered_dataset.reset_index(drop=True)

,filepath,hash,saved_time,SampleNumber,ImageID,PatientID,Modality,SeriesInstanceUID,StudyInstanceUID,ReferencedSeriesUID,...,mask.bbox.min_coord,mask.bbox.max_coord,mask.feret_diameter,mask.roundness,mask.flatness,mask.elongation,mask.equivalent_spherical_radius,mask.equivalent_spherical_perimeter,mask.equivalent_ellipsoid_diameters,mask.volume_count
0,LUNG1-001_0000/RTSTRUCT_35578236/ROI__[GTV-1]....,7afa5f960c8113372e0ad0a8e8303f7eaea396f4,2025-07-24:20:06:02,0,ROI__[GTV-1],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(290, 226, 65)","(388, 317, 86)",95.647976,0.634964,1.335891,1.367931,33.747307,14311.597300,"(50.12645370811163, 66.96349744548279, 91.6014...",1.0
1,LUNG1-001_0000/RTSTRUCT_35578236/ROI__[Spinal-...,cf1c2a5eb41e692152b3288edf6e6ed22cfec98c,2025-07-24:20:06:03,0,ROI__[Spinal-Cord],LUNG1-001,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2279381215866080725084...,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,...,"(250, 280, 27)","(278, 369, 111)",259.871631,0.560548,2.849275,6.132994,22.899470,6589.625518,"(12.449207360728114, 35.47121071739851, 217.54...",1.0
2,LUNG1-002_0001/RTSTRUCT_43245931/ROI__[GTV-1]....,cdfb2786c29b57287cc85233f0c02df51427b472,2025-07-24:20:06:20,1,ROI__[GTV-1],LUNG1-002,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2432675512669112458302...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,...,"(129, 226, 28)","(229, 325, 54)",123.299408,0.731734,1.167976,1.133250,44.090186,24428.326986,"(76.26187390348498, 89.07202002328074, 100.940...",1.0
3,LUNG1-002_0001/RTSTRUCT_43245931/ROI__[Spinal-...,3cab461010443ebd203a250988ce6e6a45d04b8f,2025-07-24:20:06:19,1,ROI__[Spinal-Cord],LUNG1-002,RTSTRUCT,1.3.6.1.4.1.32722.99.99.2432675512669112458302...,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,...,"(251, 235, 0)","(280, 334, 111)",339.654273,0.504264,3.285958,6.526126,24.922869,7805.593481,"(12.067986928374351, 39.65490376055276, 258.79...",1.0
